In [0]:
GALAXY = 'umi'

OBS_PATH = '/datascope/subaru/data/cmdfit/dSph'
HSC_FILE = '/datascope/subaru/data/cmdfit/dSph/umi_tpall3e_g24.cat'
GAIA_FILE = '/datascope/subaru/data/cmdfit/dSph/gaia.h5'
SKY_FILE = '/datascope/subaru/data/cmdfit/dSph/sky_ursaminor.feather'
FLUXSTD_FILE = '/datascope/subaru/data/cmdfit/dSph/fluxstd_ursaminor.feather'
MLCLASS_FILE = '/datascope/subaru/data/targeting/dSph/umi/ursaminor_mlclass.csv'
PMAP_FILE = '/datascope/subaru/data/cmdfit/run/umi/sim/nobin_chab_nb_250k_001/pmap.h5'
OUTPUT_PATH = '/datascope/subaru/user/dobos/netflow'
ISOCHRONES_PATH = '/datascope/subaru/data/cmdfit/isochrones/dartmouth/import/afep0_cfht_sdss_hsc'

GAIA_CROSSMATCH_RADIUS = 0.1    # in arcsec

NVISITS = 1
OUTPUT_PATH = f'/datascope/subaru/user/dobos/netflow/{GALAXY}_{NVISITS}_visit'

In [0]:
import os, sys
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.patches import Ellipse, Circle
from matplotlib.gridspec import GridSpec
from scipy.special import logsumexp
from scipy.interpolate import interp1d

from astropy import wcs
from astropy import units as u
from astropy.coordinates import Angle, SkyCoord
from astropy.time import Time

In [0]:
plt.rc('font', size=6) #controls default text size

In [0]:
if 'debug' not in globals():
    import debugpy
    debugpy.listen(('0.0.0.0', 5698))
    debug = True

In [0]:
%load_ext autoreload

In [0]:
%autoreload 2

# Compile data

## Plot definitions

In [0]:
from pfs.ga.targeting.targets.dsph import *
from pfs.ga.targeting.instrument import *
from pfs.ga.targeting.diagram import CMD, CCD, FOV, FP, ColorAxis, MagnitudeAxis
from pfs.ga.targeting.photometry import Photometry, Magnitude, Color
from pfs.ga.targeting.projection import WcsProjection
from pfs.ga.targeting.netflow import Netflow, Pointing

In [0]:
galaxy = GALAXIES[GALAXY]
hsc = galaxy.get_photometry()
cmd = galaxy.get_cmd()
ccd = galaxy.get_ccd()
gaia_cmd = galaxy.get_cmd(Gaia)

In [0]:
pointings = galaxy.get_pointings(SubaruPFI)
pointing = pointings[0]
pointings

In [0]:
wcs = WcsProjection(pointing, proj='TAN')
wfc = SubaruWFC(pointing)
fov = FOV(projection=wcs)
fp = FP(wfc)

## Load isochrones

In [0]:
from pfs.ga.isochrones.isogrid import IsoGrid

In [0]:
iso = IsoGrid()
iso.load(os.path.join(ISOCHRONES_PATH, 'isochrones.h5'))

In [0]:
iso.values.keys()

## Load probability map

In [0]:
from pfs.ga.targeting import ProbabilityMap
from pfs.ga.targeting.selection import ProbabilityCut, ProbabilitySampling

In [0]:
pmap = ProbabilityMap(cmd.axes)
pmap.load(PMAP_FILE)

In [0]:
f, axs = plt.subplots(1, 2, figsize=(6, 4), dpi=120)

l0 = cmd.plot_probability_map(axs[0], pmap, 0)
axs[0].set_title("non-member")

l1 = cmd.plot_probability_map(axs[1], pmap, 1)
axs[0].set_title("member")

f.tight_layout()

In [0]:
# Define a cut based on probability

probcut = ProbabilityCut(pmap, 1, np.log(0.001))

## Load observations

In [0]:
from pfs.ga.targeting.io import TextObservationReader
from pfs.ga.targeting.util.astro import *

In [0]:
obs = SubaruHSC.text_observation_reader().read(HSC_FILE)
obs.data.shape

In [0]:
obs.data.columns

### Plot observations with color cuts

In [0]:
cmap = plt.get_cmap('tab10')

mask = galaxy.get_selection_mask(obs, nb=True)

f = plt.figure(figsize=(8, 3), dpi=240)
gs = f.add_gridspec(1, 3, width_ratios=[2, 2, 3], wspace=0.4)   

ax = f.add_subplot(gs[0])
cmd.plot_observation(ax, obs, c='lightgray')
cmd.plot_observation(ax, obs, c='r', mask=mask, cmap=cmap)

ax = f.add_subplot(gs[1])
ccd.plot_observation(ax, obs, c='lightgray')
ccd.plot_observation(ax, obs, c='r', mask=mask, cmap=cmap)

ax = f.add_subplot(gs[2], projection=wcs.wcs)
fov.plot_observation(ax, obs, c='lightgray')
fov.plot_observation(ax, obs, c='r', mask=mask, cmap=cmap)

f.suptitle('Stars selected based on color cuts.')

## Get GAIA data

In [0]:
from pfs.ga.targeting.io import Hdf5ObservationReader, Hdf5ObservationWriter, GaiaReader

In [0]:
if os.path.isfile(GAIA_FILE):
    print('GAIA data file found, read from local.')
    r = Hdf5ObservationReader()
    gaia = r.read(GAIA_FILE, GALAXY, 'gaia')
    gaia.frame = 'icrs'
    gaia.equinox = 'J2015'
    print(gaia.data.shape)
else:
    print('GAIA data file not found, running query against archive.')
    r = GaiaReader()
    gaia = r.cone_search(galaxy.pos, galaxy.rad)
    print(gaia.data.shape)

    w = Hdf5ObservationWriter()
    w.write(gaia, GAIA_FILE, GALAXY, 'gaia')

In [0]:
gaia.data.columns

In [0]:
# Cross-match HSC with GAIA
gaia_idx, separation = obs.cross_match(gaia)
print('median separation', np.median(separation.arcsec))
hsc_gaia_mask = (separation.arcsec < GAIA_CROSSMATCH_RADIUS)
print(hsc_gaia_mask.sum())

In [0]:
# Cross-match HSC with GAIA
hsc_idx, separation = gaia.cross_match(obs)
print('median separation', np.median(separation.arcsec))
gaia_hsc_mask = (separation.arcsec < GAIA_CROSSMATCH_RADIUS)
print(gaia_hsc_mask.sum())

In [0]:
cmap = plt.get_cmap('tab10')

f = plt.figure(figsize=(6, 3), dpi=240)
gs = f.add_gridspec(1, 2, width_ratios=[2, 3], wspace=0.4)   

ax = f.add_subplot(gs[0])
gaia_cmd.plot_observation(ax, gaia, c='lightgray')
gaia_cmd.plot_observation(ax, gaia, c='r', mask=gaia_hsc_mask, cmap=cmap)

ax = f.add_subplot(gs[1], projection=wcs.wcs)
fov.plot_observation(ax, gaia, c='lightgray')
fov.plot_observation(ax, gaia, c='r', mask=gaia_hsc_mask, cmap=cmap)

f.suptitle('GAIA stars with HSC counterparts within 1"')

In [0]:
cmap = plt.get_cmap('viridis')

f = plt.figure(figsize=(8, 3), dpi=240)
gs = f.add_gridspec(1, 3, width_ratios=[2, 2, 3], wspace=0.4)   

ax = f.add_subplot(gs[0])
cmd.plot_observation(ax, obs, c='lightgray')
cmd.plot_observation(ax, obs, c='r', mask=hsc_gaia_mask, cmap=cmap)

ax = f.add_subplot(gs[1])
ccd.plot_observation(ax, obs, c='lightgray')
ccd.plot_observation(ax, obs, c='r', mask=hsc_gaia_mask, cmap=cmap)

ax = f.add_subplot(gs[2], projection=wcs.wcs)
fov.plot_observation(ax, obs, c='lightgray')
fov.plot_observation(ax, obs, c='r', mask=hsc_gaia_mask, cmap=cmap)

f.suptitle('HSC stars with GAIA counterparts within 1"')

## Merge Gaia proper motions into HSC catalog

In [0]:
# Cross-match HSC with GAIA
hsc_gaia_idx, separation = obs.cross_match(gaia)
print('median separation', np.median(separation.arcsec))
hsc_gaia_mask = (separation.arcsec < 1)

hsc_gaia_mask.shape, hsc_gaia_mask.sum()

In [0]:
gaia.data.columns

In [0]:
columns = ['parallax', 'pm', 'pmdec', 'pmra', 'err_parallax', 'err_pmdec', 'err_pmra']
obs.merge(gaia, hsc_gaia_idx, columns=columns, mask=hsc_gaia_mask)

In [0]:
obs.data.columns

## Assign probabilities

In [0]:
# Hard cuts on magnitudes

selection = galaxy.get_selection_mask(obs, nb=True, probcut=probcut)
obs.data.shape, selection.size, selection.sum()

In [0]:
# Look up membership probability

galaxy.assign_probabilities(obs, pmap, mask=selection)
(~np.isnan(obs.data['p_member'])).sum()

In [0]:
cmap = plt.get_cmap('viridis')

mask = selection & ~np.isnan(obs.data['p_member'])

f = plt.figure(figsize=(8, 3), dpi=240)
gs = f.add_gridspec(1, 3, width_ratios=[2, 2, 4], wspace=0.4)   

ax = f.add_subplot(gs[0])
cmd.plot_observation(ax, obs, c='lightgray')
cmd.plot_observation(ax, obs, c=obs.data['p_member'][mask], mask=mask, cmap=cmap)

ax = f.add_subplot(gs[1])
ccd.plot_observation(ax, obs, c='lightgray')
ccd.plot_observation(ax, obs, c=obs.data['p_member'][mask], mask=mask, cmap=cmap)

ax = f.add_subplot(gs[2], projection=wcs.wcs)
fov.plot_observation(ax, obs, c='lightgray')
l = fov.plot_observation(ax, obs, c=obs.data['p_member'][mask], mask=mask, cmap=cmap)

f.colorbar(l, ax=ax, label='membership probability')

f.suptitle('Stars selected by color cuts, colored by membersphip probability')

In [0]:
hist, bins = np.histogram(obs.data['p_member'][mask])
plt.step(0.5 * (bins[1:] + bins[:-1]), hist, where='mid')
plt.xlabel('p')
plt.ylabel('frequency')
plt.title('Distribution of membership probability')

## Assign priorities

In [0]:
galaxy.assign_priorities(obs, mask=None) # , mask=selection)
obs.data['priority'].unique()

In [0]:
cmap = plt.get_cmap('tab10')

obs_mask = obs.data['priority'] >= 0

f = plt.figure(figsize=(8, 3), dpi=240)
gs = f.add_gridspec(1, 3, width_ratios=[2, 2, 4], wspace=0.4)   

ax = f.add_subplot(gs[0])
cmd.plot_observation(ax, obs, c='lightgray')
cmd.plot_observation(ax, obs, c=obs.data['priority'][obs_mask], mask=obs_mask, cmap=cmap)

ax = f.add_subplot(gs[1])
ccd.plot_observation(ax, obs, c='lightgray')
ccd.plot_observation(ax, obs, c=obs.data['priority'][obs_mask], mask=obs_mask, cmap=cmap)

ax = f.add_subplot(gs[2], projection=wcs.wcs)
fov.plot_observation(ax, obs, c='lightgray')
l = fov.plot_observation(ax, obs, c=obs.data['priority'][obs_mask], mask=obs_mask, cmap=cmap)

f.colorbar(l, ax=ax, label='target priority')

f.suptitle('Priority class')

In [0]:
obs_mask.size, obs_mask.sum()

In [0]:
# Plot distribution of priorities

f, ax = plt.subplots(1, 1, figsize=(3.5, 2.5), dpi=240)

hist = np.bincount(obs.data['priority'][obs_mask])
ax.bar(np.arange(hist.size - 1), hist[:-1])

# hist = np.bincount(obs.data['priority'][obs_assigned & (obs.data['priority'] >= 0) & (obs.data['priority'] < 9)])
# plt.bar(np.arange(hist.size), hist, color='r')


ax.set_title('Priority class number distribution')
ax.set_xlabel('Priority class')
ax.set_ylabel('Target count')


In [0]:
# Plot distribution of required visits

f, ax = plt.subplots(1, 1, figsize=(3.5, 2.5), dpi=240)

hist = np.bincount((np.ceil(obs.data['exp_time'][obs_mask & (obs.data['priority'][obs_mask] < 9)] / 1800.0)).astype(int))
ax.bar(np.arange(hist.size), hist)

# hist = np.bincount(obs.data['priority'][obs_assigned & (obs.data['priority'] >= 0) & (obs.data['priority'] < 9)])
# plt.bar(np.arange(hist.size), hist, color='r')


ax.set_title('Distribution of required visit (t_exp = 1800 s)')
ax.set_xlabel('Number of required visits')
ax.set_ylabel('Target count')


## Load sky and flux standards

In [0]:
from pfs.ga.targeting.io import FeatherSkyReader, FeatherFluxStdReader

In [0]:
r = FeatherSkyReader()
sky = r.read(SKY_FILE)
sky.data.shape

In [0]:
r = FeatherFluxStdReader()
fluxstd = r.read(FLUXSTD_FILE)
fluxstd.data.shape

In [0]:
cmap = plt.get_cmap('tab10')

f = plt.figure(figsize=(3, 3), dpi=240)
gs = f.add_gridspec(1, 1)

ax = f.add_subplot(gs[0], projection=wcs.wcs)
fov.plot_observation(ax, obs, c='b')
fov.plot_observation(ax, sky, c='lightgray')

ax.set_title(f'Sky positions (count={len(sky)})')

# f.tight_layout()

In [0]:
cmap = plt.get_cmap('tab10')

f = plt.figure(figsize=(3, 3), dpi=240)
gs = f.add_gridspec(1, 1)

ax = f.add_subplot(gs[0], projection=wcs.wcs)
fov.plot_observation(ax, obs, c='lightgray')
fov.plot_observation(ax, fluxstd, c='r')

ax.set_title(f'Flux standards, count={len(fluxstd)}')

# f.tight_layout()

# Define the netflow problem

In [0]:
# r = Angle(50, u.arcmin)

# Filter flux standards around pointing
# obs_mask = galaxy.get_selection_mask(obs, nb=True)
# obs_mask &= obs.cone_search(p.pos, r)
# obs_mask[obs_mask] &= (obs.data['priority'][obs_mask] >= 0) & \
#                       (obs.data['priority'][obs_mask] <= 9) & \
#                       ~np.isnan(obs.data['exp_time'][obs_mask])

print('obs_mask', obs_mask.size, obs_mask.sum())

In [0]:
# Reduce number of sky fibers
sky_mask = np.full(sky.shape, True)
# sky_mask = sky.cone_search(p.pos, r)
# sky_mask &= sky.random_sample(0.3)

print('sky_mask', sky_mask.size, sky_mask.sum())

In [0]:
# Filter flux standards around pointing
fluxstd_mask = np.full(fluxstd.shape, True)
# fluxstd_mask = fluxstd.cone_search(p.pos, r)

print('fluxstd_mask', fluxstd_mask.size, fluxstd_mask.sum())

In [0]:
cmap = plt.get_cmap('tab10')

f = plt.figure(figsize=(3, 3), dpi=240)
gs = f.add_gridspec(1, 1)

ax = f.add_subplot(gs[0], projection=wcs.wcs)
# fov.plot_observation(ax, obs, c='lightgray')
fov.plot_observation(ax, obs, c='b', mask=obs_mask)
fov.plot_observation(ax, sky, c='lightgray', mask=sky_mask)
fov.plot_observation(ax, fluxstd, c='r', mask=fluxstd_mask)

for p in galaxy.get_pointings(SubaruPFI)[:]:
    SubaruPFI(SubaruWFC(p)).plot_focal_plane(ax, fov, corners=True)

# f.tight_layout()

In [0]:
ncobras = SubaruPFI().bench.cobras.nCobras
ncobras

In [0]:
obs.data.columns

In [0]:
# Construct the netflow problem

gurobi_options = dict(
    seed=0,                 # random seed
    presolve=2,             # agressiveness of presolve which tries to eliminate variables from the LP problem
    method=3,               # 3 means concurrent, 4 means deterministic concurrent
    degenmoves=0,           # degenerate simplex moves, set to 0 to prevent too much time to be spent on trying to improve the current solution
    heuristics=0.5,         # how much of the time to spend by performing heuristics
    mipfocus=1,             # mipfocus=1 is balanced toward finding more feasible solutions
                            # mipfocus=2 is balanced toward proving that the current solution is the best
                            # mipfocus=3 is to be used when the objection bound is moving very slowly
    mipgap=0.01,            # relative stopping criterion for bounds on the objective
    LogToConsole=1,         # 
    timelimit=300           # in sec
)

target_classes = {
    'sky': dict(
        prefix = 'sky',
        min_targets = 240,
        max_targets = 320,
        non_observation_cost = 0,
    ),
    'cal': dict(
        prefix = 'cal',
        min_targets = 40,
        max_targets = 240,
        non_observation_cost = 0,
        calib = True,
    ),
}
    
for i in range(10):
    target_classes[f'sci_P{i}'] = dict(
        prefix = 'sci',
        min_targets = None,
        max_targets = None,
        non_observation_cost = max(20 - 2 * i, 1),
        partial_observation_cost = 1e5,
    )

target_classes[f'sci_P0']['non_observation_cost'] = 1000
target_classes[f'sci_P1']['non_observation_cost'] = 500
target_classes[f'sci_P2']['non_observation_cost'] = 200
target_classes[f'sci_P3']['non_observation_cost'] = 100
target_classes[f'sci_P4']['non_observation_cost'] = 100
target_classes[f'sci_P5']['non_observation_cost'] = 100
target_classes[f'sci_P6']['non_observation_cost'] = 100
target_classes[f'sci_P7']['non_observation_cost'] = 50
target_classes[f'sci_P8']['non_observation_cost'] = 10
target_classes[f'sci_P9']['non_observation_cost'] = 0
    
########
    
cobra_groups = {
        'location': dict(
            groups = np.random.randint(4, size=ncobras),
            target_classes = [ 'sky' ],
            min_targets = 40,
            max_targets = 80,
            non_observation_cost = 10,
        ),
        'instrument': dict(
            groups = np.random.randint(4, size=ncobras),
            target_classes = [ 'sky' ],
            min_targets = 10,
            max_targets = 25,
            non_observation_cost = 10,
        )
    }

#######

netflow_options = dict(
    # Add a penalty if the target is too close to a black dot
    black_dot_penalty = None,
    # black_dot_penalty = lambda dist: 0,

    fiber_non_allocation_cost = 1e5,

    collision_distance = 2.0,
    elbow_collisions = True,
    # forbidden_targets = [
    #     43218108431
    # ],
    # forbidden_pairs = [
    #     [43486543901, 43218108431],
    # ],

    target_classes = target_classes,
    cobra_groups = cobra_groups,

    # time_budgets = {
    #     'science': dict(
    #         target_classes = [ 'sci_P0', 'sci_P1', 'sci_p2' ],
    #         budget = 5  # hr
    #     )
    # },

    # Do not penalize cobra moves with respect to cobra center
    cobra_move_cost = lambda dist: 0,

    num_reserved_fibers = 0,

    # This will only be used when netflow is rewritten to step-by-step fiber assignment
    # constrain_already_observed = False,

    # Allow more visits than minimally required
    allow_more_visits = True,

    epoch = 2016, # all catalogs must match
    ignore_proper_motion = False,

    # FPI configuration
    fiberids_path = '/home/dobos/project/Subaru-PFS/pfs_utils/data/fiberids'
)

debug_options = dict(
    ignoreEndpointCollisions = False,
    ignoreElbowCollisions = False,
    ignoreForbiddenPairs = False,
    ignoreForbiddenSingles = False,
    ignoreCalibTargetClassMinimum = False,
    ignoreCalibTargetClassMaximum = False,
    ignoreScienceTargetClassMinimum = False,
    ignoreScienceTargetClassMaximum = False,
    ignoreTimeBudget = False,
    ignoreCobraGroupMinimum = False,
    ignoreCobraGroupMaximum = False,
    ignoreReservedFibers = False,
)

pointings = []
exp_time = 12 / NVISITS * 900

# Observation time is the next day from now at midnight
obs_time = datetime.now() + pd.Timedelta(days=1)
obs_time = obs_time.replace(hour=0, minute=0, second=0, microsecond=0)
obs_time = Time(obs_time)
print('obs_time', obs_time)

for p in galaxy.get_pointings(SubaruPFI)[:]:
    # Need to convert targeting.pointing to netflow.pointing
    pointings.append(Pointing(
        ra=p.ra,
        dec=p.dec,
        posang=p.posang,
        obs_time=Time(datetime.now()),
        exp_time = exp_time,
        nvisits=NVISITS
    ))

nf = Netflow(f'{GALAXY}', pointings, workdir=OUTPUT_PATH,
             netflow_options=netflow_options,
             solver_options=gurobi_options,
             debug_options=debug_options)

# nf.append_science_targets(obs, mask=(obs.data['priority'] <= 9), exp_time=visits * p.exp_time)    ## All targets with priority
# nf.append_science_targets(obs, mask=(obs.data['priority'] <= 9))    ## All targets with priority
# nf.append_science_targets(obs, mask=obs_mask)    ## Only selected targets with priority
# nf.append_science_targets(obs, mask=obs_mask, exp_time=visits * p.exp_time)
# nf.append_science_targets(obs, mask=(obs.data['priority'] <= 9), exp_time=visits * p.exp_time)

nf.append_science_targets(obs, mask=obs_mask)
nf.append_sky_targets(sky, mask=sky_mask)
nf.append_fluxstd_targets(fluxstd, mask=fluxstd_mask)

len(nf.targets)

In [0]:
nf.targets.columns

In [0]:
import cProfile
profiler = cProfile.Profile()
profiler.enable()

In [0]:
nf.build()

In [0]:
profiler.disable()

In [0]:
profiler.print_stats(sort='cumtime')

In [0]:
nf.save_problem()

In [0]:
nf._Netflow__solver_options['timelimit'] = 900

In [0]:
nf.solve()

In [0]:
nf.save_solution()

### Extract results

In [0]:
assignments = nf.get_target_assignments(include_target_columns=True)
assignments

In [0]:
# Plot focal plane coordinates

f, axs = plt.subplots(1, 2, figsize=(6, 3), dpi=240)

axs[0].plot(assignments['fp_x'], assignments['fp_y'], 'o', ms=1, markeredgewidth=0)
axs[0].plot(assignments['fp_x'][100], assignments['fp_y'][100], 'or', ms=1, markeredgewidth=0)
axs[0].set_aspect('equal', adjustable='datalim')

axs[1].plot(assignments['RA'], assignments['Dec'], 'o', ms=1, markeredgewidth=0)
axs[1].plot(assignments['RA'][100], assignments['Dec'][100], 'or', ms=1, markeredgewidth=0)
# axs[1].set_aspect('equal', adjustable='datalim')

In [0]:
# Number of unique calibration targets for each visit
assignments[assignments['prefix'] == 'cal'].groupby('visit_idx')['targetid'].nunique()

In [0]:
# Number of unique sky positions for each visit
assignments[assignments['prefix'] == 'sky'].groupby('visit_idx')['targetid'].nunique()

In [0]:
# Number of observed unique science targets for each visit
assignments[assignments['prefix'] == 'sci'].groupby('visit_idx')['targetid'].nunique()

In [0]:
# Total number of unique science targets observed
assignments[assignments['prefix'] == 'sci']['targetid'].nunique()

In [0]:
summary = nf.get_target_assignment_summary()

In [0]:
# All targets observed at least once
summary[summary['num_visits'] > 0]

In [0]:
# Targets that are partially observed
summary[(summary['num_visits'] < summary['req_visits']) & (summary['num_visits'] > 0)]

In [0]:
# Science targets that are observed longer than the required number of visits
summary[(summary['prefix'] == 'sci') & (summary['num_visits'] > summary['req_visits'])]

In [0]:
# Number of unique targets per priority class
summary[summary['num_visits'] > 0].groupby('class')['targetid'].nunique()

In [0]:
# Number of missed science targets per priority class
summary[(summary['prefix'] == 'sci') & (summary['num_visits'] == 0)].groupby('class')['targetid'].nunique()

In [0]:
# Number of partially observed science targets per priority class
summary[(summary['prefix'] == 'sci') & 
        (summary['num_visits'] < summary['req_visits']) & 
        (summary['num_visits'] > 0)].groupby('class')['targetid'].nunique()

In [0]:
cobra_assignments = nf.get_cobra_assignments()
cobra_assignments

In [0]:
# Number of unassigned fibers
print('Unassigned fibers:')
for i, a in enumerate(cobra_assignments):
    print(i, (a == -1).sum())

In [0]:
# Fibers that have moved between targets
# Note: when scheduling multiple pointings, the fiber assignment is done for each pointing separately
cobra_moved = np.any(np.diff(np.stack(cobra_assignments, axis=-1), axis=-1) != 0, axis=-1)
cobra_moved, cobra_moved.sum(), np.where(cobra_moved)

In [0]:
# total # of science targets assigned

obs_assigments, obs_fiberids = nf.get_catalog_assignments_masks(obs)

for i, a in enumerate(obs_assigments):
    print(i, a.sum())

In [0]:
# total # of sky fibers assigned

sky_assigments, sky_fiberids = nf.get_catalog_assignments_masks(sky)

for i, a in enumerate(sky_assigments):
    print(i, a.sum())

In [0]:
# total # of calib targets assigned

fluxstd_assigments, fluxstd_fiberids = nf.get_catalog_assignments_masks(fluxstd)

for i, a in enumerate(fluxstd_assigments):
    print(i, a.sum())

# Look at the ILP constraints to find any issues

In [0]:
model = nf._Netflow__problem._GurobiProblem__model
constrs = model.getConstrs()
len(constrs)

In [0]:
len(nf._Netflow__constraints.all)

In [0]:
q = 0
for i in range(len(constrs)):
    name = constrs[i].ConstrName
    sense = constrs[i].Sense
    slack = constrs[i].Slack
    rhs = constrs[i].RHS

    #if slack == 0.0 and rhs > 1:
    if sense != '=' and rhs > 1:
        print(name, sense, rhs, slack)

        q += 1
        if q == 300:
            break

In [0]:
# Edges adding significant terms to the cost
# There shouldn't be any target sinks here, cause that would mean
# partially observed targets which we want to avoid

cost = nf._Netflow__problem._GurobiProblem__cost
n = cost.size()

print('total cost', cost.getValue())

for i in range(cost.size()):
    var = cost.getVar(i)
    name = var.VarName
    value = var.X
    coeff = cost.getCoeff(i)
    if value != 0 and coeff > 0:
        print(name, value, coeff, value * coeff)

# Get assignments

In [0]:
# Get assignments
assignment_masks, _ = nf.get_catalog_assignments_masks(obs)
obs_assigned = np.any(np.stack(assignment_masks, axis=-1), axis=-1)

In [0]:
# Plot distribution of priorities

f, ax = plt.subplots(1, 1, figsize=(3.5, 2.5), dpi=240)

hist = np.bincount(obs.data['priority'][obs_mask])
ax.bar(np.arange(hist.size - 1), hist[:-1])

hist = np.bincount(obs.data['priority'][obs_assigned & (obs.data['priority'] >= 0) & (obs.data['priority'] < 9)])
ax.bar(np.arange(hist.size), hist, color='r')


ax.set_title('Priority class number distribution')
ax.set_xlabel('Priority class')
ax.set_ylabel('Target count')


In [0]:
# Plot distribution of required visits

f, ax = plt.subplots(1, 1, figsize=(3.5, 2.5), dpi=240)

hist = np.bincount((np.ceil(obs.data['exp_time'][obs_mask & (obs.data['priority'][obs_mask] < 9)] / 1800.0)).astype(int))
ax.bar(np.arange(hist.size), hist)

hist = np.bincount((np.ceil(obs.data['exp_time'][obs_assigned & (obs.data['priority'] >= 0) & (obs.data['priority'] < 9)] / 1800.0)).astype(int))
ax.bar(np.arange(hist.size), hist, color='r')


ax.set_title('Distribution of required visit (t_exp = 1800 s)')
ax.set_xlabel('Number of required visits')
ax.set_ylabel('Target count')


In [0]:
# Make a plot that shows the difference of a single design vs moving fiber design

In [0]:
np.bincount(obs.data['exp_time'][(obs.data['priority'] >= 0) & (obs.data['priority'] < 9)] / 1800)

In [0]:
np.bincount(np.ceil(obs.data['exp_time'][(obs.data['priority'] >= 0) & (obs.data['priority'] < 9)] / 1800))

In [0]:
hist = np.bincount(np.ceil(obs.data['exp_time'][(obs.data['priority'] >= 0) & (obs.data['priority'] < 9)] / 1800))
plt.bar(np.arange(hist.size), hist)

hist = np.bincount(np.ceil(obs.data['exp_time'][obs_assigned & (obs.data['priority'] >= 0) & (obs.data['priority'] < 9)] / 1800))
plt.bar(np.arange(hist.size), hist, color='r')

plt.title('Distribution of required visit')

In [0]:
cmap = plt.get_cmap('tab10')

f = plt.figure(figsize=(8, 3), dpi=240)
gs = f.add_gridspec(1, 3, width_ratios=[2, 2, 4], wspace=0.4)   

axs = [None, None, None]
ax = axs[0] = f.add_subplot(gs[0])
cmd.plot_observation(ax, obs, c='lightgray')

ax = axs[1] = f.add_subplot(gs[1])
ccd.plot_observation(ax, obs, c='lightgray')

ax = axs[2] = f.add_subplot(gs[2], projection=wcs.wcs)
fov.plot_observation(ax, obs, c='lightgray')

for i, a in enumerate(obs_assigments):
    cmd.plot_observation(axs[0], obs, c=obs.data['priority'][obs_assigments[i]], mask=obs_assigments[i], cmap=cmap)
    ccd.plot_observation(axs[1], obs, c=obs.data['priority'][obs_assigments[i]], mask=obs_assigments[i], cmap=cmap)
    l = fov.plot_observation(axs[2], obs, c=obs.data['priority'][obs_assigments[i]], size=0.5, mask=obs_assigments[i], cmap=cmap)

f.colorbar(l, ax=axs[2], label='target priority')

f.suptitle(f'Observed stars')

In [0]:
cmap = plt.get_cmap('tab10')

f = plt.figure(figsize=(8, 3), dpi=240)
gs = f.add_gridspec(1, 3, width_ratios=[2, 2, 3], wspace=0.4)   

ax = f.add_subplot(gs[2], projection=wcs.wcs)
fov.plot_observation(ax, obs, c='lightgray')

for i, a in enumerate(sky_assigments):
    
    fov.plot_observation(ax, sky, c='b', mask=sky_assigments[i], size=0.5, cmap=cmap)
    fov.plot_observation(ax, fluxstd, c='r', mask=fluxstd_assigments[i], size=0.5, cmap=cmap)
    fov.plot_observation(ax, obs, c='k', mask=obs_assigments[i], size=0.2, cmap=cmap)

f.suptitle(f'Fiber assignments')

In [0]:
cmap = plt.get_cmap('tab10')

for i, a in enumerate(obs_assigments):
    f = plt.figure(figsize=(8, 3), dpi=240)
    gs = f.add_gridspec(1, 3, width_ratios=[2, 2, 3], wspace=0.4)   

    ax = f.add_subplot(gs[0])
    cmd.plot_observation(ax, obs, c='lightgray')
    cmd.plot_observation(ax, obs, c=obs.data['priority'][obs_assigments[i]], mask=obs_assigments[i], cmap=cmap)

    ax = f.add_subplot(gs[1])
    ccd.plot_observation(ax, obs, c='lightgray')
    ccd.plot_observation(ax, obs, c=obs.data['priority'][obs_assigments[i]], mask=obs_assigments[i], cmap=cmap)

    ax = f.add_subplot(gs[2], projection=wcs.wcs)
    fov.plot_observation(ax, obs, c='lightgray')
    fov.plot_observation(ax, obs, c=obs.data['priority'][obs_assigments[i]], mask=obs_assigments[i], cmap=cmap)

    f.suptitle(f'Pointing {i}')

In [0]:
cmap = plt.get_cmap('tab10')

for i, a in enumerate(sky_assigments):
    f = plt.figure(figsize=(8, 3), dpi=240)
    gs = f.add_gridspec(1, 3, width_ratios=[2, 2, 3], wspace=0.4)   

    ax = f.add_subplot(gs[2], projection=wcs.wcs)
    fov.plot_observation(ax, obs, c='lightgray')
    fov.plot_observation(ax, obs, c=obs.data['priority'][obs_assigments[i]], mask=obs_assigments[i], size=0.5, cmap=cmap)

    f.suptitle(f'Pointing {i}')

In [0]:
cmap = plt.get_cmap('tab10')

f = plt.figure(figsize=(8, 3), dpi=240)
gs = f.add_gridspec(1, 3, width_ratios=[2, 2, 3], wspace=0.4)   

ax = f.add_subplot(gs[2], projection=wcs.wcs)
fov.plot_observation(ax, obs, c='lightgray')

for i, a in enumerate(sky_assigments):
    fov.plot_observation(ax, obs, c=obs.data['priority'][obs_assigments[i]], mask=obs_assigments[i], size=0.5, cmap=cmap)

f.suptitle(f'Pointing {i}')

In [0]:
cmap = plt.get_cmap('tab10')

for i, a in enumerate(sky_assigments):
    f = plt.figure(figsize=(8, 3), dpi=240)
    gs = f.add_gridspec(1, 3, width_ratios=[2, 2, 3], wspace=0.4)   

    # ax = f.add_subplot(gs[0])
    # cmd.plot_observation(ax, obs, c='lightgray')
    # cmd.plot_observation(ax, obs, c=obs.data['priority'][sky_assigments[i]], mask=obs_assigments[i], cmap=cmap)

    # ax = f.add_subplot(gs[1])
    # ccd.plot_observation(ax, obs, c='lightgray')
    # ccd.plot_observation(ax, obs, c=obs.data['priority'][obs_assigments[i]], mask=obs_assigments[i], cmap=cmap)

    ax = f.add_subplot(gs[2], projection=wcs.wcs)
    fov.plot_observation(ax, obs, c='lightgray')
    fov.plot_observation(ax, sky, c='b', mask=sky_assigments[i], size=2, cmap=cmap)
    fov.plot_observation(ax, fluxstd, c='r', mask=fluxstd_assigments[i], size=2, cmap=cmap)
    fov.plot_observation(ax, obs, c='k', mask=obs_assigments[i], size=0.5, cmap=cmap)

    f.suptitle(f'Pointing {i}')

In [0]:
cmap = plt.get_cmap('tab10')

f = plt.figure(figsize=(8, 3), dpi=240)
gs = f.add_gridspec(1, 3, width_ratios=[2, 2, 3], wspace=0.4)   

ax = f.add_subplot(gs[2], projection=wcs.wcs)
fov.plot_observation(ax, obs, c='lightgray')

for i, a in enumerate(sky_assigments):
    
    fov.plot_observation(ax, sky, c='b', mask=sky_assigments[i], size=0.5, cmap=cmap)
    fov.plot_observation(ax, fluxstd, c='r', mask=fluxstd_assigments[i], size=0.5, cmap=cmap)
    fov.plot_observation(ax, obs, c='k', mask=obs_assigments[i], size=0.5, cmap=cmap)

f.suptitle(f'Pointing {i}')

In [0]:
# Find cobras that have to move between exposures
cobra_assignments = nf.get_cobra_assignments()

In [0]:
cobra_moves = np.sign(np.abs(np.diff(np.stack(cobra_assignments, axis=-1), axis=-1)))
cobra_moves

In [0]:
# All cobras, moving cobras
cobra_moves.shape[0], np.sum(np.any(cobra_moves == 1, axis=-1))

# Generate pfsDesign files

In [0]:
galaxy.ID

In [0]:
nf.visits

In [0]:
# Get the assignments

assignments = nf.get_target_assignments(include_target_columns=True,
                                        include_unassigned_fibers=True,
                                        include_engineering_fibers=True)

assignments.shape

In [0]:
assignments.columns

### Calculate the fluxes for the observations

The design files need the fluxes which are not processed though netflow, so we need to join them here.

Fluxes are represented as a list of numbers, one list for each target, along with a list of filter names.

Filter names are postfixed with the filter system, e.g. "r_ps1".

pfsDesign expects three difference fluxes:
* PSF flux
* Total flux
* Fiber flux

We are going to store the lists in the DataFrame we pass to `get_pfsDesign`

In [0]:
# The HSC dSph data files only contain PSF magnitudes, we need to calculate the fluxes
# This will create columns like `obs_flux_hsc_g`, 'err_flux_hsc_g', etc.
obs.calculate_flux(force=False)

In [0]:
obs.data.columns

In [0]:
# Join the HSC data with the assignments
assignments_obs = obs.data.set_index('objid').join(assignments[['targetid']].set_index('targetid'), how='inner')

# Convert index back to `objid` column
assignments_obs.reset_index(inplace=True, names='objid')

# Set additional, missing columns
assignments_obs['epoch'] = 'J2000.0'
assignments_obs['tract'] = 0
assignments_obs['patch'] = '0,0'
assignments_obs['catid'] = 15001
assignments_obs['proposalid'] = 'SSP_GA_dSph'

assignments_obs

In [0]:
for c in assignments_obs.columns:
    print(c, assignments_obs[c].dtype)

In [0]:
# Convert flux columns into columns of lists

def flux_to_list(row, prefix='obs'):
    return [ row[f'{prefix}_hsc_{b}'] for b in [ 'g', 'i', 'nb515'] ]

def filter_to_list(row):
    return [ 'hsc_g', 'hsc_i', 'hsc_nb515' ]

for prefix in [ 'psf', 'fiber', 'total' ]:
    assignments_obs[f'{prefix}_flux'] = assignments_obs.apply(lambda row: flux_to_list(row, 'obs_flux'), axis=1)
    assignments_obs[f'{prefix}_flux_err'] = assignments_obs.apply(lambda row: flux_to_list(row, 'err_flux'), axis=1)

assignments_obs['filter'] = assignments_obs.apply(filter_to_list, axis=1)

In [0]:
assignments_obs.columns

### Convert the columns of flux standards

In [0]:
fluxstd.data.columns

In [0]:
# Join the flux standards with the assignments to work with a smaller dataset
assignments_fluxstd = fluxstd.data.set_index('objid').join(assignments[['targetid']].set_index('targetid'), how='inner')

# Convert index back to `objid` column
assignments_fluxstd.reset_index(inplace=True, names='objid')

# Set additional, missing columns'
assignments_fluxstd['catid'] = -1
assignments_fluxstd['tract'] = 0
assignments_fluxstd['patch'] = '0,0'
assignments_fluxstd['catid'] = 15001
assignments_fluxstd['proposalid'] = ''

assignments_fluxstd

In [0]:
assignments_fluxstd[[ f'filter_{f}' for f in 'grizyj' ]]

In [0]:
# Convert flux columns into columns of lists

def flux_to_list(row, prefix='psf_flux'):
    fluxes = []
    for f in 'grizyj':
        if row[f'filter_{f}'] is not None:
            fluxes.append(row[f'{prefix}_{f}'])
    return fluxes
        

def filter_to_list(row):
    filters = []
    for f in 'grizyj':
        if row[f'filter_{f}'] is not None:
            filters.append(row[f'filter_{f}'])
    return filters

for prefix in [ 'psf', 'fiber', 'total' ]:
    assignments_fluxstd[f'{prefix}_flux'] = assignments_fluxstd.apply(lambda row: flux_to_list(row, 'psf_flux'), axis=1)
    assignments_fluxstd[f'{prefix}_flux_err'] = assignments_fluxstd.apply(lambda row: flux_to_list(row, 'psf_flux_error'), axis=1)

assignments_fluxstd['filter'] = assignments_fluxstd.apply(filter_to_list, axis=1)

In [0]:
assignments_fluxstd.columns

In [0]:
assignments_fluxstd[['filter', 'psf_flux', 'psf_flux_err', 'total_flux', 'total_flux_err']]

### List of all targets

In [0]:
# Pick only columns required for PfsDesign
columns = [ 'objid', 'epoch', 'proposalid', 'tract', 'patch', 'catid',
           'filter', 'psf_flux', 'psf_flux_err', 'fiber_flux', 'fiber_flux_err', 'total_flux', 'total_flux_err' ]

all_targets = pd.concat([ assignments_obs[columns], assignments_fluxstd[columns] ])

In [0]:
for c in all_targets.columns:
    print(c, all_targets[c].dtype)

In [0]:
all_targets

### Merge the assignments with the fluxes etc. calculated above

In [0]:
# assignments.join(obs.data[cols], on=['targetid', 'objid'], how='left')

a = assignments.set_index('targetid')
b = all_targets.set_index('objid')

# Only include columns that are not in a
cols = b.columns.difference(a.columns)
print(cols)

all_assignments = a.join(b[cols], how='left')
all_assignments.reset_index(inplace=True, names='targetid')
print(all_assignments.columns)

In [0]:
b['filter']

In [0]:
# The missing filters are the sky fibers
all_assignments[['targetid', 'filter']]

In [0]:
all_assignments

In [0]:
for c in all_assignments.columns:
    print(c, all_assignments[c].dtype)

In [0]:
# Do some final polishing

# TODO: determine tract and patch from the coordinates
all_assignments['tract'] = 0
all_assignments['patch'] = '0,0'
all_assignments['obcode'] = '-1'

all_assignments['filter'] = all_assignments['filter'].apply(lambda x: x if isinstance(x, list) else [])
for prefix in ['fiber', 'psf', 'total']:
    all_assignments[f'{prefix}_flux'] = all_assignments[f'{prefix}_flux'].apply(lambda x: x if isinstance(x, list) else [])
    all_assignments[f'{prefix}_flux_err'] = all_assignments[f'{prefix}_flux_err'].apply(lambda x: x if isinstance(x, list) else [])

In [0]:
from pfs.ga.targeting.netflow import Design

Design.get_pfsDesign_visit(nf.visits[0], all_assignments)

### Read back the file

In [0]:
from astropy.io import fits
from astropy.table import Table

# Load back the design file with astropy for direct inspection
with fits.open('pfsDesign-0x0000000000000000.fits', memmap=False) as hdul:
    hdul.info()
    # print(hdul[0].header)
    # print(hdul[1].header)
    # print(hdul[2].header)

    data = Table(hdul[1].data)
    
data.info()

In [0]:
data